![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
LangChain 101
</b></font> </br></p>

---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Colab-Umfeld</font> </br></p>
# Installierte Python Version
import sys
print(f"Python Version: ",sys.version)
# Installierte LangChain Bibliotheken
print()
print("Installierte LangChain Bibliotheken:")

!pip list | grep '^langchain'
# Unterdrückt die "DeprecationWarning" von LangChain für die Memory-Funktionden
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langsmith.client")

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  SetUp API-Keys (setup_api_keys)</font> </br></p>
def setup_api_keys():
    """Konfiguriert alle benötigten API-Keys aus Google Colab userdata"""
    from google.colab import userdata
    import os
    from os import environ

    # Dictionary der benötigten API-Keys
    keys = {
        'OPENAI_API_KEY': 'OPENAI_API_KEY',
        'HF_TOKEN': 'HF_TOKEN',
        # Weitere Keys bei Bedarf
    }

    # Keys in Umgebungsvariablen setzen
    for env_var, key_name in keys.items():
        environ[env_var] = userdata.get(key_name)

    return {k: environ[k] for k in keys.keys()}

# Verwendung
all_keys = setup_api_keys()
# Bei Bedarf einzelne Keys direkt zugreifen
# WEATHER_API_KEY = all_keys['WEATHER_API_KEY']

# 1 | Was ist LangChain?
---

LangChain ist ein Framework zur **Entwicklung von Anwendungen mit großen Sprachmodellen** (LLMs). Es vereinfacht die Integration von LLMs in eigene Anwendungen durch:

- Modulare Komponenten für verschiedene Aufgaben
- Vorgefertigte Ketten (Chains) für komplexe Workflows
- Einfache Integration externer Datenquellen
- Werkzeuge für das Speichern von Konversationskontexten



[Einführung](https://python.langchain.com/docs/introduction/)   
[Konzepte](https://python.langchain.com/docs/concepts/)   
[API-References](https://python.langchain.com/api_reference/index.html)   
[Integrations](https://python.langchain.com/docs/integrations/providers/)

---

[Tutorials](https://python.langchain.com/docs/tutorials/)   
[How-to-Guides](https://python.langchain.com/docs/how_to/)   

In [ ]:
!uv pip install --system --prerelease allow -q langchain_community langchain_openai

# 2 | Kernkonzepte
---




## 2.1 Message-Typen

Beim Einsatz von Large Language Models (LLMs) wie GPT gibt es drei Hauptarten von Nachrichten (Messages), die die Interaktion mit dem Modell steuern. Diese Messages definieren, wie das Modell reagiert, welche Eingaben es erhält und welche Ausgaben es generiert.



**System Message:**   
Die `System Message` dient dazu, das Verhalten des LLMs zu steuern. Sie ist eine nicht sichtbare Nachricht, die dem Modell Anweisungen gibt, wie es sich verhalten soll. Dabei kann sie die Persönlichkeit, den Tonfall und die Einschränkungen des Modells beeinflussen. Die Systemnachricht funktioniert wie eine Art "Rollenspiel-Anweisung", um das Modell in eine gewünschte Rolle zu versetzen.

*Merkmale:*
- Definiert, wie das Modell reagieren soll
- Legt Persönlichkeit, Verhalten und Einschränkungen fest
- Wird zu Beginn einer Sitzung gesetzt und bleibt bestehen
- Nicht sichtbar für den Benutzer

*Beispiel einer System Message:*
```json
{
  "role": "system",
  "content": "Du bist ein KI-Assistent, der präzise und informative Antworten in einem professionellen Ton liefert."
}
```
Dieses Beispiel weist das Modell an, in einem professionellen Tonfall kurze und informative Antworten zu geben.

---



**User Message**   
Die `User Message` ist die eigentliche Eingabe des Benutzers. Sie kann eine Frage, eine Aufforderung, kontextrelevanten Inhalt oder ein Befehl sein. Das Modell nutzt diese Nachricht als Ausgangspunkt für die Generierung einer Antwort.

*Merkmale:*
- Direkte Eingabe des Benutzers
- Kann eine Frage, Aufforderung, kontextrelevanten Inhalt oder einen Befehl enthalten
- Basis für die Antwort des Modells

*Beispiel einer User Message:*
```json
{
  "role": "user",
  "content": "Was sind die wichtigsten Unterschiede zwischen KI und maschinellem Lernen?"
}
```
Hier fragt der Benutzer nach den Unterschieden zwischen KI und maschinellem Lernen.

---


**AI Message**   
Die `AI Message` ist die Antwort, die das Modell generiert. Sie basiert auf der User Message und den Anweisungen aus der System Message. Die AI Message kann verschiedene Eigenschaften haben, ist aber hauptsächlich auf den Inhalt fokussiert.

*Merkmale:*
- Antwort des Modells auf die Benutzeranfrage
- Kann verschiedene Eigenschaften haben (z. B. Länge, Stil)
- Hauptsächlich inhaltlich relevant

*Beispiel einer AI Message:*
```json
{
  "role": "assistant",
  "content": "Künstliche Intelligenz (KI) ist ein weites Feld, das verschiedene Teilgebiete umfasst, darunter auch das maschinelle Lernen (ML). ML konzentriert sich auf die Erstellung von Modellen, die aus Daten lernen und Vorhersagen treffen."
}
```
Hier gibt das Modell eine inhaltliche Antwort auf die Frage des Benutzers.

---



**Zusammenhang**   
Die drei Message-Typen arbeiten zusammen, um eine sinnvolle Interaktion mit dem Modell zu ermöglichen:

+  **System Message** legt die Regeln und das Verhalten des Modells fest.
+ **User Message** stellt eine Anfrage oder einen Befehl.
+ **AI Message** gibt die generierte Antwort basierend auf den vorherigen Nachrichten.

**Beispiel für eine vollständige Konversation:**    
```json
[
  {
    "role": "system",
    "content": "Sie sind ein KI-Assistent, der präzise und informative Antworten in einem professionellen Ton liefert."
  },
  {
    "role": "user",
    "content": "Kannst der erklären, was große Sprachtmodelle sind?"
  },
  {
    "role": "assistant",
    "content": "Sicher! Large Language Models (LLMs) sind fortschrittliche KI-Systeme, die mit riesigen Textdatenmengen trainiert werden. Sie können menschenähnliche Texte verstehen und generieren und sind daher für verschiedene Anwendungen wie Chatbots, Content-Erstellung und mehr nützlich."
  }
]
```
In diesem Beispiel wurde das Modell so eingestellt, dass es in einem freundlichen Tonfall antwortet, dann erhält es eine Benutzeranfrage und gibt eine passende Antwort.

---



In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown

In [ ]:
messages = [
    SystemMessage(
        content="Du bist ein kompetenter und hilfreicher Assistent."
    ),
    HumanMessage(
        content="Nenne mir die Hauptstadt von Usbekistan?"
    ),
]

In [ ]:
type(messages)

In [ ]:
MODEL = 'gpt-4o-mini'
TEMPERATURE = 0.0

llm = ChatOpenAI(model=MODEL, temperature= TEMPERATURE)
response = llm.invoke(messages)

In [ ]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(response.content))

Die Liste `messages` wird um den  der letzen Abfrage und eine Nachfrage ergänzt

In [ ]:
messages.append(response)
messages.append(HumanMessage(content="Bis Du sicher? Ich glaube, sie wurde kürzlich umbenannt."))

In [ ]:
for i, message in enumerate(messages, 1):
    print(f"{i}. {message}")

Die Konversationsliste `messages` wird wieder an das Modell übergeben.

In [ ]:
response = llm.invoke(messages)

In [ ]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(response.content))

**Fazit**    
Die System-, User- und AI-Message sind essenzielle Bestandteile jeder Interaktion mit einem LLM. Während die System Message das Verhalten vorgibt, stellt die User Message die eigentliche Anfrage dar, auf die das Modell mit einer AI Message antwortet. Dieses Konzept ist zentral für den Einsatz von LLMs in Chatbots, KI-gestützten Assistenten und anderen interaktiven Anwendungen.



## 2.2 Prompts

Prompts sind die Eingaben bzw. Aufträge an das LLM. LangChain bietet verschiedene Template-Systeme:

<p><font color='black' size="5">
Simple Prompt
</font></p>

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

In [ ]:
# 1. Einfaches Template mit Platzhaltern
simple_explanation_prompt = PromptTemplate.from_template(
    "Erkläre {thema} in einfachen Worten."
)

In [ ]:
print(simple_explanation_prompt.format(thema="Machine Learning"))

In [ ]:
thema = "Generative KI"
prompt = simple_explanation_prompt.format(thema=thema)
response = llm.invoke(prompt)

In [ ]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(response.content))


<p><font color='black' size="5">
Eingabeaufforderungsvorlagen
</font></p>



Mit LangChain kann man Operationsketten erstellen, die als Teil einer LLM-fähigen Anwendung ausgeführt werden. Eine dieser Operationen ist eine Eingabeaufforderungsvorlage, mit der man Text in eine zuvor erstellte Eingabeaufforderung einfügen können. In diesem Beispiel wird eine Eingabeaufforderungsvorlage erstellt, die das Modell auffordert, einen zufälligen Titel für einen Blogbeitrag zu erstellen.

```
Gibt nur den Titel eines Blog-Beitragsartikels zum Thema {topic} in {language} zurück.
```

Um dieses Ziel zu erreichen, verwenden wir ein **PromptTemplate**-Objekt.

In [ ]:
topic = "Haustiere für Datenwissenschaftler"
language = "german"

In [ ]:
blog_title_prompt = PromptTemplate.from_template(
    "Gib nur den Titel eines Blogbeitrags zum Thema {topic} in {language} zurück."
)

In [ ]:
# Höhere Temperaturen für mehr Kreativität
llm = ChatOpenAI(model=MODEL, temperature=0.7)

# Verkettung von Prompt und LLM -  Erläuterungen zu Verkettungen siehe weiter unten
title_chain = blog_title_prompt | llm

# Aufrufen der Kette mit Eingaben
response = title_chain.invoke({'topic': topic, 'language': language})

In [ ]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(response.content))

<p><font color='black' size="5">
Chat Prompt
</font></p>

In [ ]:
chat_template = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher und humorvoller Assistent."),
    ("human", "Erkläre mir {thema}"),
    ("assistant", "Ich erkläre dir {thema} gerne.")
])

In [ ]:
print(chat_template.format(thema="Machine Learning"))

In [ ]:
# Verkettung von Prompt und LLM
chat_chain = chat_template | llm

# Aufrufen der Kette mit Eingaben
thema = "Machine Learning"
response = chat_chain.invoke({'thema': thema})

In [ ]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(response.content))

<p><font color='black' size="5">
Few-Shot-Learning Prompt
</font></p>

In [ ]:
examples = [
    {"frage": "Was ist Python?", "antwort": "Eine interpretierende Programmiersprache."},
    {"frage": "Was ist Java?", "antwort": "Eine objektorientierte Sprache."}
]

In [ ]:
# Prompt für einzelne Beispiele
example_prompt = PromptTemplate.from_template(
    "Frage: {frage}\nAntwort: {antwort}"
)

In [ ]:
# Few-Shot-Prompt
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Beantworte die Fragen basierend auf den Beispielen:",
    suffix="Frage: {input}\nAntwort:",
    input_variables=["input"]
)

In [ ]:
print(few_shot_prompt.format(input="Was ist Cobol?"))

In [ ]:
# geringere Temperaturen für mehr Sachlichkeit
llm = ChatOpenAI(model=MODEL, temperature=0.2)

# Verkettung von Prompt und LLM
few_shot_chain = few_shot_prompt | llm

# Aufrufen der Kette mit Eingaben
response = few_shot_chain.invoke({'input': "Was ist Cobol"})

In [ ]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(response.content))

## 2.3 Modelle

LangChain unterstützt verschiedene LLMs und bietet flexible Konfigurationsmöglichkeiten:

In [ ]:
from langchain_openai import ChatOpenAI, OpenAI

In [ ]:
# Text-Completion-Modell (z. B. für Aufgaben wie Zusammenfassungen)
completion_model = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    temperature=0.7,
    max_tokens=500
)

# Chat-Modell für interaktive Konversationen
chat_model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.9
)

<p><font color='black' size="5">
Vergleich ChatOpenAI vs OpenAI
</font></p>

| Aspekt                 | `ChatOpenAI`                                           | `OpenAI`                                              |
|-------------------------|--------------------------------------------------------|-------------------------------------------------------|
| Typ                     | Chat-Model Wrapper                                     | Text-Completion Wrapper                               |
| Hauptzweck              | Gesprächsbasierte Interaktion (multi-turn Dialoge)     | Klassische Textvervollständigung (prompt → completion) |
| Basis-Modelle           | GPT-Modelle mit Chat-API (`gpt-3.5-turbo`, `gpt-4`, etc.) | GPT-Modelle über Completion-API (`text-davinci-003`, etc.) |
| Eingabeformat           | Nachrichtenstruktur (`messages`)                      | Flacher Textprompt (`prompt`)                         |
| Rückgabeformat          | Strukturiert (Nachricht/Text, Rolleninformationen)     | Einfacher Text (reiner Completion-Text)               |
| Kosten                  | Günstiger (bei GPT-3.5, GPT-4 Turbo)                   | Teurer (bei vergleichbarem Modell `text-davinci-003`)  |
| Zukunftsperspektive     | **Empfohlen für neue Projekte**                        | Veraltet, wird seltener empfohlen                     |
| Zusätzliche Funktionen  | Bessere Steuerung (z.B. Systemnachrichten, Tool-Use)    | Klassische Vervollständigung ohne Rollen oder Kontext  |

## 2.4 OutputParser

Ein OutputParser ist ein Konzept in LangChain, das hilft, die Antworten eines Modells richtig weiterzuverarbeiten. Es nimmt die rohe Ausgabe (zum Beispiel einen langen Text) und formt sie in ein bestimmtes Format, das später in der Anwendung leichter benutzt werden kann.

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Einfacher Prompt ohne explizite Formatvorgabe
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent. Antworte bitte im JSON-Format mit den Feldern 'name' und 'alter'."),
    ("human", "Bitte gib Name und Alter einer Person an.")
])

# Modell
chat_model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Struktur der Antwort definieren
response_schemas = [
    ResponseSchema(name="name", description="Der vollständige Name der Person."),
    ResponseSchema(name="alter", description="Das Alter der Person in Jahren.")
]

# StructuredOutputParser erstellen
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# LCEL-Chain
chain = prompt | chat_model | output_parser

# Ausführen
response = chain.invoke({"input": ""})

print(response)

## 2.5 Chains (LCEL)

Chains verbinden mehrere Komponenten zu einer Verarbeitungskette. LangChain Expression Language (LCEL) ist das **neue Programmiermodell in LangChain v0.3+**, das die Entwicklung von LLM-Anwendungen vereinfacht.



**Hier die Kernpunkte:**

1. Was ist LCEL?
```python
# LCEL nutzt den Pipe-Operator (|) für klare Verkettungen
chain = prompt | model | output_parser
```
- Eine deklarative Sprache zum Verketten von LangChain-Komponenten
- Ermöglicht linearen Datenfluss zwischen Komponenten
- Basiert auf dem Pipe-Operator (|) für intuitive Verbindungen

2. Warum LCEL nutzen?
- Bessere Lesbarkeit des Codes
- Einfachere Wartung und Debugging
- Verbesserte Performance durch optimierte Ausführung
- Bessere Typsicherheit und Fehlererkennung
- Unterstützt modernes Streaming und Async-Operationen

3. Praktisches Beispiel:

<p><font color='black' size="5">
Simple Chain
</font></p>

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Prompt-Template
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher und humorvoller Assistent."),
    ("human", "{input}")
])

#  Einfache LCEL-Kette
chain = prompt | chat_model | StrOutputParser()

# Ausführung
response = chain.invoke({"input": "Erkläre LangChain Expression Language."})

In [ ]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(response))


LCEL ist der empfohlene Weg für alle neuen LangChain-Projekte, da es die Entwicklung vereinfacht und zukunftssicher macht.

<p><font color='black' size="5">
Sequential Chains
</font></p>

Sequentielle Chain: Linear, ein Input wird schrittweise verarbeitet

In [ ]:
# Prompts
zusammenfassen_template = ChatPromptTemplate.from_messages([
    ("system", "Fasse den folgenden Text prägnant zusammen."),
    ("human", "{text}")
])

uebersetzen_template = ChatPromptTemplate.from_messages([
    ("system", "Übersetze den folgenden Text ins Deutsche."),
    ("human", "{text}")
])

In [ ]:
# Verarbeitungskette

# Erste Verarbeitungskette: Zusammenfassung
summarization_chain = (
    zusammenfassen_template
    | chat_model
    | StrOutputParser()
    | (lambda output: {"text": output})  # Notwendig, für String-> Dict, wird von chat_model erwartet
)

# Zweite Verarbeitungskette: Übersetzung
translation_chain = (
    uebersetzen_template
    | chat_model
    | StrOutputParser()
)

# Gesamte Kette: Zusammenfassen und dann Übersetzen
sequential_chain = summarization_chain | translation_chain

In [ ]:
# Input Text
text = """
Machine Learning is a specialized branch of artificial intelligence (AI) dedicated to the development of algorithms and systems that can automatically learn from data and past experiences. Instead of following rigid, pre-defined instructions, machine learning models identify patterns, adapt their behavior, and improve their performance over time based on the information they are exposed to. This ability enables them to make predictions, recognize complex relationships, and solve problems in dynamic environments. Machine learning techniques are widely applied across various fields, including healthcare, finance, transportation, and entertainment, driving innovations such as personalized recommendations, autonomous vehicles, and intelligent diagnostics.
"""

In [ ]:
# Aufruf der Kette
response = sequential_chain.invoke({"text": text})

In [ ]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(response))

<p><font color='black' size="5">
Q&A-Chain
</font></p>

QA Chain: Kontextbasiert, kombiniert mehrere Inputs für eine einzelne Verarbeitung

In [ ]:
# Q&A Prompt
qa_template = ChatPromptTemplate.from_messages([
    ("system", "Beantworte die Frage basierend auf dem gegebenen Kontext."),
    ("human", """
    Kontext: {context}

    Frage: {question}
    """)
])

In [ ]:
# Verkettung
qa_chain = qa_template | chat_model | StrOutputParser()

In [ ]:
# Inhalte
context = """
Python ist eine beliebte Programmiersprache für Machine Learning.
Sie bietet viele Bibliotheken wie TensorFlow und PyTorch.
"""
question = "Welche ML-Bibliotheken gibt es für Python?"

In [ ]:
# Aufruf
response = qa_chain.invoke({
    "context": context,
    "question": question
})

In [ ]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(response))

## 2.6 Runnables



<p><font color='black' size="5">
Was sind Runnables?
</font></p>

Runnables sind das Fundament der modernen LangChain-Architektur (ab Version 0.2+). Sie stellen eine einheitliche Schnittstelle für alle Komponenten dar, die Eingaben verarbeiten und Ausgaben erzeugen können. Runnables können wie Funktionen aufgerufen werden, bieten jedoch zusätzliche Vorteile:

- **Einheitliches Interface**: Alle Runnables teilen dieselben grundlegenden Methoden (invoke, batch, stream, etc.)
- **Kompositionsfähigkeit**: Runnables können einfach miteinander kombiniert werden (LCEL nutzt dieses Konzept)
- **Flexibilität**: Sie unterstützen synchrone und asynchrone Operationen
- **Streaming**: Einfache Unterstützung für die schrittweise Ausgabe

Fast alle wichtigen Komponenten in modernem LangChain sind Runnables, darunter:
- Prompts (PromptTemplate, ChatPromptTemplate)
- Modelle (ChatOpenAI, OpenAI)
- OutputParser
- Chains
- Tools
- Retriever


<p><font color='black' size="5">
Eigene Runnables erstellen
</font></p>

Man kann auch eigene Runnables erstellen, indem man eine Klasse definiert, die das Runnable-Interface implementiert oder eine Funktion mit `RunnableLambda` wrapped:


In [ ]:
from langchain_core.runnables import RunnableLambda

# 1. Mit RunnableLambda eine einfache Transformation erstellen
def text_verdoppeln(eingabe):
    return f"{eingabe} - {eingabe}"

verdoppler = RunnableLambda(text_verdoppeln)

# Verwendung des eigenen Runnables
print(verdoppler.invoke("Hallo"))  # Ausgabe: Hallo-Hallo

In [ ]:
# 2. In eine Kette einbauen
runnable_chain_mit_verdoppler = prompt | model | parser | verdoppler

# Test der Kette
ergebnis = runnable_chain_mit_verdoppler.invoke({"eingabe": "Nenne ein kurzes Wort."})
print(f"Ergebnis mit Verdoppler: {ergebnis}")

<p><font color='black' size="5">
LCEL und Runnables
</font></p>

Die LangChain Expression Language (LCEL) basiert vollständig auf dem Runnable-Konzept. Der Pipe-Operator (`|`) verbindet Runnables miteinander, indem er die Ausgabe eines Runnables zur Eingabe des nächsten macht. Jede LCEL-Kette ist selbst ein Runnable.


In [ ]:
# Beispiel einer LCEL-Kette mit verschiedenen Runnable-Typen
from langchain_core.runnables import RunnableParallel

# Eine Runnable-Komponente, die parallele Berechnungen durchführt
parallel_runnable = RunnableParallel(
    zusammenfassung=prompt | model | parser,
    wortanzahl=lambda eingabe: len(eingabe["eingabe"].split())
)

# Verwenden der parallelen Komponente
ergebnis = parallel_runnable.invoke({"eingabe": "Wie funktionieren Runnables in LangChain?"})
print("\nPARALLELE VERARBEITUNG:")
print(f"Wortanzahl: {ergebnis['wortanzahl']}")
print(f"Zusammenfassung: {ergebnis['zusammenfassung'][:50]}...")


<p><font color='black' size="5">
Runnable-Methoden
</font></p>

Jedes Runnable unterstützt standardmäßig folgende Methoden:

1. **invoke()**: Für einzelne, synchrone Anfragen
2. **batch()**: Verarbeitet mehrere Eingaben parallel
3. **stream()**: Gibt Teilergebnisse zurück, sobald sie verfügbar sind
4. **ainvoke()**: Asynchrone Version von invoke
5. **abatch()**: Asynchrone Version von batch
6. **astream()**: Asynchrone Version von stream



Hier ein einfaches Beispiel mit den verschiedenen Aufrufmethoden:


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser

# Einfaches Template erstellen
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher und humorvoller Assistent."),
    ("human", "{eingabe}")
])

# Modell definieren
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Output-Parser
parser = StrOutputParser()

# Runnable-Kette erstellen
runnable_chain = prompt | model | parser

In [ ]:
# 1. invoke - Einzelne Anfrage
ergebnis = runnable_chain.invoke({"eingabe": "Was ist ein Runnable in LangChain?"})
print(f"INVOKE-ERGEBNIS:\n{ergebnis}\n")

In [ ]:
# 2. batch - Mehrere Anfragen parallel
ergebnisse = runnable_chain.batch([
    {"eingabe": "Erkläre den Begriff LCEL kurz."},
    {"eingabe": "Was sind die Hauptvorteile von Runnables?"}
])
print("BATCH-ERGEBNISSE:")
for i, ergebnis in enumerate(ergebnisse, 1):
    print(f"Anfrage {i}: {ergebnis[:50]}...\n")

In [ ]:
# 3. stream - Schrittweise Ausgabe
print("STREAM-ERGEBNIS:")
for chunk in runnable_chain.stream({"eingabe": "Zähle von 1 bis 5 auf. Mache eine Pause von 2 Sekunden nach jedem Schritt."}):
    print(chunk, end="")
print("\n")

# 3 | Überblick LangChain Konzepte
---

Konzepte von LangChain sind grundlegende Bausteine und Prinzipien, die das Framework ausmachen und seine Funktionsweise ermöglichen. Hier sind einige der wichtigsten Konzepte:

+ Chat-Modelle (**Chat models**): LLMs, die über eine Chat-API verfügbar sind und Sequenzen von Nachrichten verarbeiten1.
+ Nachrichten (**Messages**): Kommunikationseinheiten in Chat-Modellen für Ein- und Ausgabe1.
+ Chat-Verlauf (**Chat history**): Eine Sequenz von Nachrichten, die eine Konversation darstellt.
+ Tools (**Tools**): Funktionen mit definierten Schemata für Name, Beschreibung und Argumente.
+ Strukturierte Ausgabe (**Structured output**): Technik, um Chat-Modelle in strukturierten Formaten antworten zu lassen.
+ Retrieval Augmented Generation (**RAG**): Technik zur Verbesserung von Sprachmodellen durch Kombination mit externen Wissensbasen.
+ Prompt-Vorlagen (**Prompt template**s): Komponenten zur Erstellung strukturierter Prompts für LLMs.
+ Chains (**Chains**): Verknüpfungen mehrerer LLMs oder anderer Komponenten für komplexere Anwendungen.
+ Agenten (**Agents**): Nutzen Sprachmodelle, um Aktionssequenzen auszuwählen und mit externen Ressourcen zu interagieren.
+ Retriever (**Retriever**): Komponenten, die relevante Dokumente aus einer Wissensbasis abrufen.

<p><font color='black' size="5">
Anbieterbezogene APIs in LangChain
</font></p>

| API-Name in LangChain | Anbieter             | Bemerkung                                 |
|-----------------------|----------------------|-------------------------------------------|
| `ChatOpenAI`           | OpenAI               | Chat-Modelle (`gpt-3.5-turbo`, `gpt-4`, ...) |
| `OpenAI`               | OpenAI               | Text Completion (`text-davinci-003`, alt)  |
| `ChatAnthropic`        | Anthropic            | Claude-Modelle (`Claude 3`, etc.)          |
| `ChatGoogleGenerativeAI` | Google (Gemini)      | Gemini-Modelle (`gemini-1.5-pro`, etc.)    |
| `ChatCohere`           | Cohere               | Command-Modelle                           |
| `ChatMistralAI`        | Mistral              | Mistral-Modelle (`Mistral 7B`, Mixtral etc.) |
| `ChatFireworks`        | Fireworks AI         | Zugriff auf viele Open-Weight-Modelle      |
| `ChatAzureOpenAI`      | Azure (Microsoft)    | OpenAI-Modelle über Azure API              |
| `BedrockLLM`           | AWS Bedrock          | Zugang zu Claude, Titan, Jurassic, etc.    |

# A | Aufgabe
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.


<p><font color='black' size="5">
Erstellen einer LangChain-Kette zur Textanalyse
</font></p>

Entwickeln Sie eine Verarbeitungskette, die einen Text analysiert und verschiedene Informationen darüber extrahiert.

**Schritte:**
1. Erstelle ein ChatPromptTemplate, das ein LLM anweist, einen Text zu analysieren
2. Die Analyse soll folgende Aspekte umfassen:
   - Hauptthema des Textes
   - Tonalität (formal, informell, etc.)
   - Schlüsselwörter (5-10)
   - Kurze Zusammenfassung (max. 3 Sätze)
3. Formatiere die Ausgabe strukturiert mit Markdown-Überschriften
4. Teste die Kette mit mindestens zwei verschiedenen Texten




<p><font color='black' size="5">
Few-Shot-Learning für Textklassifikation
</font></p>

Erstellen Sie ein System, das mithilfe von Few-Shot-Learning Texte in vorgegebene Kategorien klassifiziert.

**Schritte:**
1. Definiere 3-5 Kategorien für die Klassifikation (z.B. Sport, Politik, Technologie, Kultur)
2. Erstelle einen FewShotPromptTemplate mit Beispielen für jede Kategorie
3. Entwickle eine Chain, die neue Texte klassifiziert
4. Implementiere eine Funktion, die neben der Kategorie auch eine Begründung für die Einordnung liefert
5. Teste das System mit verschiedenen Texten, die nicht in den Beispielen vorkommen


<p><font color='black' size="5">
Q&A-System mit Sequential Chain
</font></p>

Entwickeln Sie ein System, das Fragen zu einem gegebenen Kontext beantwortet, aber zuerst den Kontext zusammenfasst und dann die Frage beantwortet.


**Schritte:**

+ Erstellen Sie zwei Templates:
    + Ein Template zur Zusammenfassung des Kontexts
    + Ein Template zur Beantwortung einer Frage basierend auf der Zusammenfassung
+ Verbinden Sie die Templates in einer sequentiellen Kette
+ Implementieren Sie einen Mechanismus, der die Zusammenfassung und die Antwort getrennt zurückgibt
+ Nutzen Sie LCEL (LangChain Expression Language) für die Verkettung
+ Testen Sie das System mit verschiedenen Kontexten und Fragen